In [1]:
import pandas as pd      

## Loading three files 

In [2]:
companies=pd.read_csv('C:\\Users\\Administrator\\Downloads\\datasets\\Teclov\\companies.txt',sep='\t',encoding='ANSI')

In [3]:
companies.head(2)

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/Organization/-Fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/Organization/-Qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014


In [4]:
maping=pd.read_csv('C:\\Users\\Administrator\\Downloads\\datasets\\Teclov\\mapping.csv',sep=',',encoding='ANSI')

In [5]:
maping

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,NaN,0,1,0,0,0,0,0,0,0
1,3D,0,0,0,0,0,1,0,0,0
2,3D Printing,0,0,0,0,0,1,0,0,0
3,3D Technology,0,0,0,0,0,1,0,0,0
4,Accounting,0,0,0,0,0,0,0,0,1
5,Active Lifestyle,0,0,0,0,1,0,0,0,0
6,Ad Targeting,0,0,0,0,0,0,0,0,1
7,Advanced Materials,0,0,0,0,0,1,0,0,0
8,Adventure Travel,1,0,0,0,0,0,0,0,0
9,Advertising,0,0,0,0,0,0,0,0,1


In [6]:
rounds=pd.read_csv('C:\\Users\\Administrator\\Downloads\\datasets\\Teclov\\rounds2.csv',encoding='ANSI')

In [7]:
rounds.head(2)

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN


### Cleaning perma link in companies file

In [8]:
a=companies['permalink'].apply( lambda x: x.replace('/Organization/',('')) )


In [9]:
b=a.apply( lambda x: x.replace('-','')) .str.lower().str.strip()


#### Unique in  permalink

In [10]:
b.nunique()

66290

### Droping the pemelink and adding the new column comapny_name

In [11]:
new_companies=companies.drop(['permalink','name'],axis=1)

In [12]:
new_companies['company_name']=b.str.lower()


In [13]:
new_companies.head(2) # new df for companies file after filtering the names.

,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,company_name
0,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,fame
1,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,qounter


## Cleaning company_permalink in round2 file

In [14]:
a1=rounds['company_permalink'].apply(lambda x: x.replace('/organization/',''))
a2=a1.apply(lambda x: x.replace('/ORGANIZATION/',''))
rounds_new=a2.apply(lambda x: x.replace('-','')).str.lower().str.strip()

#### unique company name in round2 file

In [15]:
rounds_new.nunique()

66290

### Droping the pemelink and adding the new column comapny_name

In [16]:
new_round2=rounds.drop(['company_permalink','funding_round_permalink'],axis=1)

In [17]:
new_round2['company_name']=rounds_new.str.lower()

In [18]:
new_round2.head(2) # new df for round2 file after filtering the names.

,funding_round_type,funding_round_code,funded_at,raised_amount_usd,company_name
0,venture,B,05-01-2015,10000000.0,fame
1,venture,A,14-10-2014,NaN,qounter


### merging the the two new data frames 

In [19]:
master_frame=pd.merge(new_round2,new_companies)
master_frame.shape

(115200, 13)

### converging to only companies currently operating

In [20]:
master_frame_on=master_frame[master_frame['status']!='closed']
master_frame_on.shape

(106449, 13)

#### converging dataframe to ['venture','seed','private_equity','angel']

In [21]:
master_frame_conv=master_frame_on[master_frame_on.funding_round_type.isin (['venture','seed','private_equity','angel'])]
master_frame_conv.shape

(87039, 13)

##### improvising data with amount

### calculation of Representative Values  ['venture','seed','private_equity','angel']


In [22]:
mean_venture=master_frame_conv[master_frame_conv.funding_round_type.isin (['venture'])]


In [23]:
mean_venture['raised_amount_usd'].mean()


11899609.197291862

In [24]:
mean_venture['raised_amount_usd'].count()

46748

In [25]:
mean_seed=master_frame_conv[master_frame_conv.funding_round_type.isin (['seed'])]

In [26]:
mean_seed.mean()

raised_amount_usd    738753.403386
dtype: float64

In [27]:
mean_seed['raised_amount_usd'].count()

21798

In [28]:
mean_private_equity=master_frame_conv[master_frame_conv.funding_round_type.isin (['private_equity'])]


In [29]:
mean_private_equity.mean()

raised_amount_usd    7.550415e+07
dtype: float64

In [30]:
mean_private_equity['raised_amount_usd'].count()

1808

In [31]:
mean_angel=master_frame_conv[master_frame_conv.funding_round_type.isin (['angel'])]

In [32]:
mean_angel.mean()

raised_amount_usd    978893.929515
dtype: float64

In [33]:
mean_angel['raised_amount_usd'].count()

4313

# Choosing the venture stratgy

In [34]:
mean_venture=master_frame_conv[master_frame_conv.funding_round_type.isin (['venture'])]
mean_venture.shape

(51581, 13)

## top9 countries

In [35]:

top_country=mean_venture.pivot_table(index='country_code',values='raised_amount_usd',aggfunc='sum').sort_values(ascending=False,by='raised_amount_usd').head(9)
top_country

,raised_amount_usd
country_code,
USA,4.008023e+11
CHN,3.910177e+10
GBR,1.834651e+10
IND,1.409556e+10
CAN,8.839931e+09
FRA,6.775044e+09
ISR,6.503021e+09
DEU,6.061141e+09
JPN,3.248032e+09


In [36]:
#has to be done while time of purchasing the company

# improvising the condition ('5000000<= raised_amount_usd <=15000000')
#df = df.query('5000000<= raised_amount_usd <=15000000')


In [37]:
master_frame.head(2)

,funding_round_type,funding_round_code,funded_at,raised_amount_usd,company_name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,venture,B,05-01-2015,10000000.0,fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,venture,A,14-10-2014,NaN,qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014


In [38]:
new_category=mean_venture['category_list'].str.split('|',n=1,expand=True)



In [39]:
main_venture=mean_venture.drop(['category_list'],axis=1)

In [40]:
main_venture['primary_sector']=new_category[0]

In [41]:
#now the main data frame is # main_venture

## Problem 5
### merging the data frames main_venture & mapping

In [42]:
main_sector=pd.merge(main_venture,maping,left_on='primary_sector',right_on='category_list')

### Applying the conditions '5000000<=raised_amount_usd<=15000000'

In [43]:
main_sector=main_sector.query('5000000<=raised_amount_usd<=15000000')

In [44]:
main_sector = main_sector.rename(columns={'category_list':'Primary_list'})

In [81]:
map1=pd.melt(maping,id_vars=['category_list'],value_vars=['Automotive & Sports','Blanks','Cleantech / Semiconductors','Entertainment','Health','Manufacturing','News, Search and Messaging','Others','Social, Finance, Analytics, Advertising'])


In [46]:
map1=map1.query('value==1')

In [47]:
map1=map1.rename(columns = {'variable':'Major_sector'}) 


### merging the map file with the main sector file

In [48]:
main_frame=pd.merge(main_sector,map1,left_on='Primary_list',right_on='category_list')
main_frame

,funding_round_type,funding_round_code,funded_at,raised_amount_usd,company_name,homepage_url,status,country_code,state_code,region,...,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising",category_list,Major_sector,value
0,venture,B,05-01-2015,10000000.0,fame,http://livfame.com,operating,IND,16,Mumbai,...,0,1,0,0,0,0,0,Media,Entertainment,1
1,venture,NaN,06-10-2015,15000000.0,90min,http://www.90min.com,operating,GBR,H9,London,...,0,1,0,0,0,0,0,Media,Entertainment,1
2,venture,NaN,07-05-2013,5800000.0,90min,http://www.90min.com,operating,GBR,H9,London,...,0,1,0,0,0,0,0,Media,Entertainment,1
3,venture,A,06-08-2014,5000000.0,alldefdigital,http://alldefdigital.com,operating,USA,CA,Los Angeles,...,0,1,0,0,0,0,0,Media,Entertainment,1
4,venture,A,26-02-2015,5000000.0,chefsfeed,http://www.chefsfeed.com,operating,USA,CA,SF Bay Area,...,0,1,0,0,0,0,0,Media,Entertainment,1
5,venture,A,01-08-2006,5000000.0,huffingtonpost,http://www.huffingtonpost.com,acquired,USA,NY,New York City,...,0,1,0,0,0,0,0,Media,Entertainment,1
6,venture,B,01-09-2007,5000000.0,huffingtonpost,http://www.huffingtonpost.com,acquired,USA,NY,New York City,...,0,1,0,0,0,0,0,Media,Entertainment,1
7,venture,B,13-05-2015,5000000.0,mediabong,http://www.mediabong.com,operating,USA,NY,New York City,...,0,1,0,0,0,0,0,Media,Entertainment,1
8,venture,A,01-09-2015,6000000.0,moseeker,http://www.moseeker.com,operating,CHN,23,NaN,...,0,1,0,0,0,0,0,Media,Entertainment,1
9,venture,NaN,08-01-2010,12500000.0,newscorporation,http://www.newscorp.com,ipo,USA,NY,New York City,...,0,1,0,0,0,0,0,Media,Entertainment,1


## United States Of America

In [49]:
D1=main_frame.query("country_code=='USA' ")   # data frame for United States Of America
D1.shape


(10482, 26)

In [50]:
count=D1.groupby('Major_sector')[['company_name']].count()
count=count.reset_index()
count=count.rename(columns={"company_name": "count"})
count

,Major_sector,count
0,Automotive & Sports,157
1,Blanks,61
2,Cleantech / Semiconductors,2146
3,Entertainment,521
4,Health,854
5,Manufacturing,740
6,"News, Search and Messaging",1451
7,Others,2735
8,"Social, Finance, Analytics, Advertising",1817


In [51]:
d1=pd.merge(D1,count)
d1.head(2)

,funding_round_type,funding_round_code,funded_at,raised_amount_usd,company_name,homepage_url,status,country_code,state_code,region,...,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising",category_list,Major_sector,value,count
0,venture,A,06-08-2014,5000000.0,alldefdigital,http://alldefdigital.com,operating,USA,CA,Los Angeles,...,1,0,0,0,0,0,Media,Entertainment,1,521
1,venture,A,26-02-2015,5000000.0,chefsfeed,http://www.chefsfeed.com,operating,USA,CA,SF Bay Area,...,1,0,0,0,0,0,Media,Entertainment,1,521


In [52]:
#sum
sumus=D1.groupby('Major_sector')[['raised_amount_usd']].sum()
sumus=sumus.reset_index()
sumus=sumus.rename(columns={'raised_amount_usd':'sum'})
sumus

,Major_sector,sum
0,Automotive & Sports,1.374304e+09
1,Blanks,5.691130e+08
2,Cleantech / Semiconductors,1.974190e+10
3,Entertainment,4.510748e+09
4,Health,7.748830e+09
5,Manufacturing,6.707809e+09
6,"News, Search and Messaging",1.287045e+10
7,Others,2.444407e+10
8,"Social, Finance, Analytics, Advertising",1.594762e+10


In [53]:
# merging to previous dataframe
D1=pd.merge(d1,sumus)
D1.head(2)

,funding_round_type,funding_round_code,funded_at,raised_amount_usd,company_name,homepage_url,status,country_code,state_code,region,...,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising",category_list,Major_sector,value,count,sum
0,venture,A,06-08-2014,5000000.0,alldefdigital,http://alldefdigital.com,operating,USA,CA,Los Angeles,...,0,0,0,0,0,Media,Entertainment,1,521,4.510748e+09
1,venture,A,26-02-2015,5000000.0,chefsfeed,http://www.chefsfeed.com,operating,USA,CA,SF Bay Area,...,0,0,0,0,0,Media,Entertainment,1,521,4.510748e+09


## United Kingdom

In [54]:
D2=main_frame.query("country_code=='GBR' ")   #data frame for United Kingdom

In [55]:
count1=D2.groupby('Major_sector')[['company_name']].count()
count1=count1.reset_index()
count1=count1.rename(columns={"company_name": "count"})
count1

,Major_sector,count
0,Automotive & Sports,16
1,Blanks,5
2,Cleantech / Semiconductors,114
3,Entertainment,52
4,Health,23
5,Manufacturing,42
6,"News, Search and Messaging",66
7,Others,137
8,"Social, Finance, Analytics, Advertising",95


In [56]:
d2=pd.merge(D2,count1)
d2

,funding_round_type,funding_round_code,funded_at,raised_amount_usd,company_name,homepage_url,status,country_code,state_code,region,...,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising",category_list,Major_sector,value,count
0,venture,NaN,06-10-2015,15000000.0,90min,http://www.90min.com,operating,GBR,H9,London,...,1,0,0,0,0,0,Media,Entertainment,1,52
1,venture,NaN,07-05-2013,5800000.0,90min,http://www.90min.com,operating,GBR,H9,London,...,1,0,0,0,0,0,Media,Entertainment,1,52
2,venture,A,13-05-2010,8800000.0,eutechnyx,http://press.eutechnyx.com,operating,GBR,E5,Gateshead,...,1,0,0,0,0,0,Games,Entertainment,1,52
3,venture,B,01-06-2011,10000000.0,mindcandy,http://www.mindcandy.com,operating,GBR,H9,London,...,1,0,0,0,0,0,Games,Entertainment,1,52
4,venture,A,23-11-2006,10860000.0,mindcandy,http://www.mindcandy.com,operating,GBR,H9,London,...,1,0,0,0,0,0,Games,Entertainment,1,52
5,venture,B,31-03-2008,5700000.0,mobango,http://www.mobango.com,operating,GBR,H9,London,...,1,0,0,0,0,0,Games,Entertainment,1,52
6,venture,B,21-06-2012,11000000.0,naturalmotion,http://naturalmotion.com,acquired,GBR,K2,London,...,1,0,0,0,0,0,Games,Entertainment,1,52
7,venture,NaN,28-05-2014,5000000.0,outplayentertainment,http://outplay.com/,operating,GBR,U3,Dundee,...,1,0,0,0,0,0,Games,Entertainment,1,52
8,venture,B,25-01-2011,10000000.0,playcastmedia,http://playcast-media.com,operating,GBR,H9,London,...,1,0,0,0,0,0,Games,Entertainment,1,52
9,venture,B,18-01-2007,9850000.0,playerx,http://www.playerx.com,operating,GBR,H9,London,...,1,0,0,0,0,0,Games,Entertainment,1,52


In [57]:
#sum
sumuk=D2.groupby('Major_sector')[['raised_amount_usd']].sum()
sumuk=sumuk.reset_index()
sumuk=sumuk.rename(columns={'raised_amount_usd':'sum'})
sumuk

,Major_sector,sum
0,Automotive & Sports,1.670516e+08
1,Blanks,4.025000e+07
2,Cleantech / Semiconductors,1.028029e+09
3,Entertainment,4.544111e+08
4,Health,2.025375e+08
5,Manufacturing,3.658243e+08
6,"News, Search and Messaging",5.585007e+08
7,Others,1.189442e+09
8,"Social, Finance, Analytics, Advertising",7.976084e+08


In [58]:
# merging to previous dataframe
D2=pd.merge(d2,sumuk)
D2.head(2)

,funding_round_type,funding_round_code,funded_at,raised_amount_usd,company_name,homepage_url,status,country_code,state_code,region,...,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising",category_list,Major_sector,value,count,sum
0,venture,NaN,06-10-2015,15000000.0,90min,http://www.90min.com,operating,GBR,H9,London,...,0,0,0,0,0,Media,Entertainment,1,52,454411059.0
1,venture,NaN,07-05-2013,5800000.0,90min,http://www.90min.com,operating,GBR,H9,London,...,0,0,0,0,0,Media,Entertainment,1,52,454411059.0


## India

In [59]:
D3=main_frame.query("country_code=='IND' ")   #data frame for India

In [60]:
count3=D3.groupby('Major_sector')[['company_name']].count()
count3=count3.reset_index()
count3=count3.rename(columns={"company_name": "count"})
count3

,Major_sector,count
0,Automotive & Sports,13
1,Blanks,2
2,Cleantech / Semiconductors,20
3,Entertainment,32
4,Health,18
5,Manufacturing,21
6,"News, Search and Messaging",51
7,Others,102
8,"Social, Finance, Analytics, Advertising",32


In [61]:
d3=pd.merge(D3,count3)

In [62]:
d3

,funding_round_type,funding_round_code,funded_at,raised_amount_usd,company_name,homepage_url,status,country_code,state_code,region,...,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising",category_list,Major_sector,value,count
0,venture,B,05-01-2015,10000000.0,fame,http://livfame.com,operating,IND,16,Mumbai,...,1,0,0,0,0,0,Media,Entertainment,1,32
1,venture,NaN,27-11-2006,5000000.0,dhruva,http://www.dhruva.com/,operating,IND,19,Bangalore,...,1,0,0,0,0,0,Games,Entertainment,1,32
2,venture,A,29-03-2007,5000000.0,games2win,http://www.games2win.com,operating,IND,16,Mumbai,...,1,0,0,0,0,0,Games,Entertainment,1,32
3,venture,B,30-03-2011,6000000.0,games2win,http://www.games2win.com,operating,IND,16,Mumbai,...,1,0,0,0,0,0,Games,Entertainment,1,32
4,venture,B,03-11-2015,5000000.0,pokkt,http://www.pokkt.com,operating,IND,16,Mumbai,...,1,0,0,0,0,0,Games,Entertainment,1,32
5,venture,NaN,12-12-2006,8000000.0,cleartrip,http://www.cleartrip.com,operating,IND,16,Mumbai,...,1,0,0,0,0,0,Travel,Entertainment,1,32
6,venture,NaN,12-05-2014,5400000.0,cleartrip,http://www.cleartrip.com,operating,IND,16,Mumbai,...,1,0,0,0,0,0,Travel,Entertainment,1,32
7,venture,A,31-07-2015,10000000.0,tripfactory,http://www.tripfactory.com,operating,IND,19,Bangalore,...,1,0,0,0,0,0,Travel,Entertainment,1,32
8,venture,A,06-06-2007,5000000.0,via3,http://in.via.com,operating,IND,19,Bangalore,...,1,0,0,0,0,0,Travel,Entertainment,1,32
9,venture,D,05-07-2012,14500000.0,yatraonline,http://www.Yatra.com,operating,IND,10,New Delhi,...,1,0,0,0,0,0,Travel,Entertainment,1,32


In [63]:
#sum
sumind=D3.groupby('Major_sector')[['raised_amount_usd']].sum()
sumind=sumind.reset_index()
sumind=sumind.rename(columns={'raised_amount_usd':'sum'})
sumind

,Major_sector,sum
0,Automotive & Sports,136900000.0
1,Blanks,27000000.0
2,Cleantech / Semiconductors,165380000.0
3,Entertainment,272080000.0
4,Health,160740000.0
5,Manufacturing,200900000.0
6,"News, Search and Messaging",423834545.0
7,Others,934289507.0
8,"Social, Finance, Analytics, Advertising",290543500.0


In [64]:
# merging to previous dataframe
D3=pd.merge(d3,sumind)
D3.head(2)

,funding_round_type,funding_round_code,funded_at,raised_amount_usd,company_name,homepage_url,status,country_code,state_code,region,...,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising",category_list,Major_sector,value,count,sum
0,venture,B,05-01-2015,10000000.0,fame,http://livfame.com,operating,IND,16,Mumbai,...,0,0,0,0,0,Media,Entertainment,1,32,272080000.0
1,venture,NaN,27-11-2006,5000000.0,dhruva,http://www.dhruva.com/,operating,IND,19,Bangalore,...,0,0,0,0,0,Games,Entertainment,1,32,272080000.0


## All data Frames

## United States(U.S)            - D1
## United Kingdom(GBR)     - D2
## India(IND)                          - D3

In [65]:
# 1. Total number of investments (count)

count['count'].sum()   #US

10482

In [66]:
count1['count'].sum()    #UK

550

In [67]:
count3['count'].sum()     #IND

291

In [68]:
#2. Total amount of investment (USD)

sumus['sum'].sum()       #US

93914840098.0

In [69]:
sumuk['sum'].sum()       #UK

4803654985.0

In [70]:
sumind['sum'].sum()      #IND

2611667552.0

 
 
 3. Top sector (based on count of investments)
  
 4. Second-best sector (based on count of investments)
   
 5. Third-best sector (based on count of investments)
 
 
 
 
  6. Number of investments in the top sector (refer to point 3)
   
  7. Number of investments in the second-best sector (refer to point 4)
   
  8. Number of investments in the third-best sector (refer to point 5)
   


In [71]:
#US
count.sort_values(ascending=False,by='count').head(3)

,Major_sector,count
7,Others,2735
2,Cleantech / Semiconductors,2146
8,"Social, Finance, Analytics, Advertising",1817


In [72]:
#UK
count1.sort_values(ascending=False,by='count').head(3)

,Major_sector,count
7,Others,137
2,Cleantech / Semiconductors,114
8,"Social, Finance, Analytics, Advertising",95


In [73]:
#IND
count3.sort_values(ascending=False,by='count').head(3)

,Major_sector,count
7,Others,102
6,"News, Search and Messaging",51
3,Entertainment,32


 9. For the top sector count-wise (point 3), which company received the highest investment?
   

 

In [74]:
#US
D1.query("Major_sector=='Others'").groupby('company_name')[['raised_amount_usd']].sum().sort_values(ascending=False,by='raised_amount_usd').head(1)

,raised_amount_usd
company_name,
virtustream,64300000.0


In [75]:
#UK
D2.query("Major_sector=='Others'").groupby('company_name')[['raised_amount_usd']].sum().sort_values(ascending=False,by='raised_amount_usd').head(1)

,raised_amount_usd
company_name,
electriccloud,37000000.0


In [76]:
#IND
D3.query("Major_sector=='Others'").groupby('company_name')[['raised_amount_usd']].sum().sort_values(ascending=False,by='raised_amount_usd').head(1)

,raised_amount_usd
company_name,
firstcrycom,39000000.0


 10. For the second-best sector count-wise (point 4), which company received the highest investment?

In [77]:
# US
D1.query("Major_sector=='Cleantech / Semiconductors'").groupby('company_name')[['raised_amount_usd']].sum().sort_values(ascending=False,by='raised_amount_usd').head(1)

,raised_amount_usd
company_name,
biodesix,75300000.0


In [78]:
# UK
D2.query("Major_sector=='Cleantech / Semiconductors'").groupby('company_name')[['raised_amount_usd']].sum().sort_values(ascending=False,by='raised_amount_usd').head(1)

,raised_amount_usd
company_name,
eusapharma,35600000.0


In [79]:
# IND
D3.query("Major_sector=='News, Search and Messaging'").groupby('company_name')[['raised_amount_usd']].sum().sort_values(ascending=False,by='raised_amount_usd').head(1)

,raised_amount_usd
company_name,
gupshuptechnologyindiapvtltd,33000000.0
